In [1]:
import tensorflow as tf
import cv2,os
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

n_classes=2
path_pos='Car/Positive'
path_neg='Car/Negative'

def read_data(path,label):
    images = []
    labels = []
    dirs = os.listdir( path )
    for files in dirs:
        file_name=path+"/"+files
        image = cv2.imread(file_name,0)
        image=np.reshape(image, 40*100)
        images.append(image)
        labels.append(label)  
    return images, labels



images_pos,labels_pos=read_data(path_pos,1)
images_neg,labels_neg=read_data(path_neg,0)

images=images_pos+images_neg
labels=labels_pos+labels_neg
#images=np.concatenate(images_pos,images_neg)
#labels=np.concatenate(labels_pos,labels_neg)

x_train, x_test,y_train,y_test = train_test_split(images, labels,test_size=0.1, random_state=41)

x_train=np.asarray(x_train)
y_train=np.asarray(y_train)

print(x_train.shape)


(945, 4000)


In [2]:
# Parameters
learning_rate = 0.001
batch_size = 20
display_step = 10
num_steps=200
# Network Parameters
n_input = 4000 
num_classes=2

x = tf.placeholder(tf.float32, [None, n_input],name="x")
y = tf.placeholder(tf.int32, [None],name="y")

In [3]:
def random_batch(x_train, y_train, batch_size):
    rnd_indices = np.random.randint(0, len(x_train), batch_size)
    x_batch = x_train[rnd_indices]
    y_batch = y_train[rnd_indices]
    return x_batch, y_batch


def conv_net(x):
    x = tf.reshape(x, shape=[-1, 40, 100, 1])
    conv1 = tf.layers.conv2d(x, 8, 5, activation=tf.nn.relu,padding="SAME")
    conv1 = tf.layers.max_pooling2d(conv1, 2, 2,padding="SAME")
    conv2 = tf.layers.conv2d(conv1, 16, 5, activation=tf.nn.relu,padding="SAME")
    conv2 = tf.layers.max_pooling2d(conv2, 2, 2,padding="SAME")
    conv3 = tf.layers.conv2d(conv2, 32, 3, activation=tf.nn.relu)
    conv3 = tf.layers.max_pooling2d(conv3, 2, 2)
    fc1 = tf.contrib.layers.flatten(conv3)
    fc1 = tf.layers.dense(fc1, 512,activation=tf.nn.relu)
    out = tf.layers.dense(fc1, num_classes,name="output")
    
    return out

In [4]:
pred = conv_net(x)

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=pred, labels=y)
cost = tf.reduce_mean(xentropy)

optimizer = tf.train.AdamOptimizer()
training_op=optimizer.minimize(cost)

correct = tf.nn.in_top_k(pred, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()


In [5]:
saver = tf.train.Saver()

sess= tf.Session() 

n_epochs = 10
n_batches = len(x_train) // batch_size

sess.run(init)
for epoch in range(n_epochs):
    for step in range(n_batches):
        batch_x, batch_y = random_batch(x_train, y_train, batch_size)
        sess.run(training_op, feed_dict={x: batch_x, y: batch_y})
    acc = sess.run( accuracy, feed_dict={x: x_train,y: y_train})
    print('Epoch:',epoch, ', Accuracy:',acc)

print("Optimization Finished!")

Epoch: 0 , Accuracy: 0.969312
Epoch: 1 , Accuracy: 0.991534
Epoch: 2 , Accuracy: 0.994709
Epoch: 3 , Accuracy: 0.990476
Epoch: 4 , Accuracy: 0.996825
Epoch: 5 , Accuracy: 1.0
Epoch: 6 , Accuracy: 1.0
Epoch: 7 , Accuracy: 1.0
Epoch: 8 , Accuracy: 1.0
Epoch: 9 , Accuracy: 1.0
Optimization Finished!


In [6]:
# saver = tf.train.Saver()

# sess= tf.Session() 

# sess.run(init)
# for step in range(1, num_steps+1):
#     batch_x, batch_y = random_batch(x_train, y_train, batch_size)
#     sess.run(training_op, feed_dict={x: batch_x, y: batch_y})
#     if step % display_step == 0 or step == 1:
#         acc = sess.run( accuracy, feed_dict={x: x_train,y: y_train})
#         print('Step:',step, ', Accuracy:',acc)

# print("Optimization Finished!")
    

In [7]:
save_path = saver.save(sess, "models/car_cnn.ckpt")

In [8]:
test_acc=sess.run(accuracy, feed_dict={x: x_test,y: y_test})
print("Testing Accuracy:", test_acc)

Testing Accuracy: 0.971429
